# An HMM demo for Turing

In [1]:
using Turing, Distributions
using PyPlot, PyCall
@pyimport matplotlib.cm as cm
@pyimport matplotlib.animation as animation
@pyimport numpy as np
@pyimport IPython.display as D

Load transition matrix T and observations obs

In [10]:
include(Pkg.dir("Turing")*"/doc/source/notebooks/data/hmmdemo.data.jl");

LoadError: LoadError: Redefiining of existing variable (obs) is not allowed.
while loading /Users/yebai/.julia/v0.5/Turing/doc/source/notebooks/data/hmmdemo.data.jl, in expression starting on line 41

Define the model

In [4]:
K = 5
N = 201
initial = fill(1.0 / K, K)
means = (collect(1.0:K)*2-K)*2

@model hmmdemo(y) = begin
    states = tzeros(Int,N)
    # T = TArray{Array{Float64,}}
    
    # Prior over T
    # for i=1:K, @assume T[i,:] ~ Dirichlet(ones(K)./K); end
    
    states[1] ~ Categorical(initial)
    for i = 2:N
        states[i] ~ Categorical(vec(T[states[i-1],:]))
        y[i] ~ Normal(means[states[i]], 4)
    end
    return states
end

hmmdemo (generic function with 4 methods)

Run the sampler, collect results

In [5]:
data = Dict(:obs=>obs);
chain = @sample(hmmdemo(obs), PG(100,100));

LoadError: UndefVarError: obs not defined

Print the 1st sample trajectory

In [10]:
chain.value[1].value[:states]'

1×201 Array{Int64,2}:
 2  2  2  1  1  1  1  1  2  2  1  1  3  …  1  1  1  3  3  1  2  2  2  2  2  2

Plot samples

In [11]:
fig, ax = plt[:subplots]();
ax[:set_xlim](( 0, 210))
ax[:set_ylim]((-20, 20))
line, = ax[:plot]([], [], lw=2)

# initialization function: plot the background of each frame
function init()
    ax[:plot](1:N, obs, "rs")
    return (line,)
end

# animation function. This is called sequentially
function animate(i)
    n = mod(i,length(chain.value))
    line[:set_data](1:N, means[chain.value[n+1].value[:states]])
    return (line,)
end

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=200, blit=true)

plt[:close]()


In [12]:
D.HTML(anim[:to_html5_video]())

PyObject <IPython.core.display.HTML object>